In [6]:
from os import listdir
from os.path import isfile, join
import os
import pandas
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
import fos_reader
import re
import math
onlyfiles = [f for f in listdir(os.getcwd()) if isfile(join(os.getcwd(), f))]

Process ForkPoolWorker-24:
Process ForkPoolWorker-17:
Process ForkPoolWorker-23:
Process ForkPoolWorker-19:
Traceback (most recent call last):
Process ForkPoolWorker-18:
Traceback (most recent call last):
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-21:
Traceback (most recent call last):
Process ForkPoolWorker-20:
Process ForkPoolWorker-22:
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 297, in _bootst

In [7]:
def reader_fos(fileName):
    df = pandas.read_json(fileName, lines=True)
    fos_with_nan_and_wages = df[["fos"]]
    fos_with_wages = fos_with_nan_and_wages.loc[fos_with_nan_and_wages["fos"].apply(lambda x: type(x) is not float)]
    fos = fos_with_wages
    fos["fos"] = fos_with_wages["fos"].apply(lambda row: [field["name"] for field in row])
    return fos

I am read in files in which author and field of study are replaces to indexes.

In [8]:
dblp_part_regex = re.compile("author_indexed_fos*")
part_files_filter_obj = filter(lambda file_name: dblp_part_regex.match(file_name), onlyfiles)

In [9]:
part_files = list(part_files_filter_obj)

In [10]:
list_of_dataframes = list(map(reader_fos, part_files))

In [11]:
df = pandas.concat(list_of_dataframes, ignore_index=True)

In [12]:
fos = df

In [16]:
foses_dict = {}
def notice_fos(name):
    number = foses_dict.get(name, 0)
    foses_dict[name] = number + 1
fos["fos"].apply(lambda x: [notice_fos(fos_number) for fos_number in x])
foses_counted = pandas.DataFrame.from_dict(foses_dict, orient='index')

0                       [None, None, None, None, None, None]
1          [None, None, None, None, None, None, None, Non...
2                 [None, None, None, None, None, None, None]
3          [None, None, None, None, None, None, None, Non...
4          [None, None, None, None, None, None, None, Non...
                                 ...                        
4074431                 [None, None, None, None, None, None]
4074432     [None, None, None, None, None, None, None, None]
4074433                       [None, None, None, None, None]
4074434    [None, None, None, None, None, None, None, Non...
4074435                 [None, None, None, None, None, None]
Name: fos, Length: 4074436, dtype: object

In [35]:
sorted_foses_counted = foses_counted.sort_values(by=[0])

Field of study are counted, sorted and choosen are only first [10:2010]

In [43]:
foses_more_frequent_than_50 = sorted_foses_counted.loc[sorted_foses_counted[0] > 50]

In [117]:
foses_300 = foses_more_frequent_than_50[:-10][-2000:]
foses_300_indexes = foses_300.index

In [118]:
filtered = fos["fos"].apply(lambda foses: list(filter(lambda fos_number: fos_number in foses_300_indexes, foses)))

Get rid of these publications, which haven't choosen field of studies.

In [119]:
filtered.apply(lambda x: x == [])

0          False
1          False
2          False
3          False
4          False
           ...  
4074431    False
4074432    False
4074433    False
4074434    False
4074435    False
Name: fos, Length: 4074436, dtype: bool

In [120]:
are_empty =  _

In [121]:
filtered

0                                           [524, 2428, 921]
1          [96, 177, 6981, 130, 1910, 2524, 4617, 4535, 9...
2                                 [524, 271, 276, 423, 1003]
3                                      [765, 109, 2048, 119]
4                     [362, 447, 419, 1188, 1501, 585, 8796]
                                 ...                        
4074431                                      [96, 2724, 330]
4074432                    [485, 447, 276, 2681, 1923, 1159]
4074433                                               [1374]
4074434                     [96, 1891, 2524, 296, 183, 2484]
4074435                                        [14, 15, 953]
Name: fos, Length: 4074436, dtype: object

In [122]:
new_filtered = filtered[are_empty.apply(lambda is_empty: not is_empty)]

In [123]:
new_filtered

0                                           [524, 2428, 921]
1          [96, 177, 6981, 130, 1910, 2524, 4617, 4535, 9...
2                                 [524, 271, 276, 423, 1003]
3                                      [765, 109, 2048, 119]
4                     [362, 447, 419, 1188, 1501, 585, 8796]
                                 ...                        
4074431                                      [96, 2724, 330]
4074432                    [485, 447, 276, 2681, 1923, 1159]
4074433                                               [1374]
4074434                     [96, 1891, 2524, 296, 183, 2484]
4074435                                        [14, 15, 953]
Name: fos, Length: 4030076, dtype: object

Creating graph from field of study. Verticles are fields of study, edges are weighted by number of times, when they exist commonly in publication.

In [251]:
graph = {}
import itertools
def add_fos_to_graph(fos_list, graph):
    for x, y in itertools.product(fos_list, fos_list):
        if x != y:
            if x < y:
                edge_name = str(x) + "_" + str(y)
            else:
                edge_name = str(y) + "_" + str(x)
            weight = graph.get(edge_name, 0)
            graph[edge_name] = weight + 1
            
new_filtered.apply(lambda fos_group: add_fos_to_graph(fos_group, graph))

KeyboardInterrupt: 

In [125]:
import networkx
ngraph = networkx.Graph()
for edge, weight in graph.items():
    nodes = edge.split("_")
    node_x = int(nodes[0])
    node_y = int(nodes[1])
    ngraph.add_edge(node_x, node_y, weight = weight)

In [126]:
networkx.write_gexf(ngraph, "fos_graph.gexf")

Saving into file. Next step is generate communities using gephi and saving them into csv.

In [139]:
foses_csv = pandas.read_csv('fos_graph_with_classes.csv')

In [144]:
classes = foses_csv[["Id", "modularity_class"]]

In [155]:
grouped_by_class = classes.groupby(["modularity_class"])["Id"].apply(list)

In [157]:
grouped_by_class.apply(len)

modularity_class
0    477
1    308
2    369
3    122
4     87
5     16
6    586
7     26
8      9
Name: Id, dtype: int64

In [158]:
with open("fos_index_map.json", "r") as fos_names_file:
    json_names_fos = fos_names_file.read()

In [168]:
import json
fos_name_to_index = json.loads(json_names_fos)
index_to_fos_name = {index: fos_name for fos_name, index in fos_name_to_index.items()}

In [171]:
classes_with_names = grouped_by_class.apply(lambda foses: [index_to_fos_name[fos_index] for fos_index in foses])

In [173]:
classes_with_names.loc[0]

['Pedagogy',
 'Knowledge management',
 'Curriculum',
 'Social psychology',
 'Psychology',
 'Affect (psychology)',
 'Cognition',
 'Probabilistic logic',
 'Visualization',
 'Multimedia',
 'Human–computer interaction',
 'Wearable computer',
 'Data visualization',
 'Wearable technology',
 'Physical therapy',
 'Information system',
 'Case-based reasoning',
 'Information and Communications Technology',
 'Developmental psychology',
 'Neuroscience',
 'Library science',
 'Analogy',
 'Online analytical processing',
 'View',
 'Data mining',
 'Knowledge extraction',
 'Informatics',
 'Higher education',
 'Data collection',
 'Health care',
 'Design methods',
 'Nearest neighbor search',
 'Inference',
 'Situation awareness',
 'Data sharing',
 'Social media',
 'Publishing',
 'Information sharing',
 'Conversation',
 'Leverage (finance)',
 'Management science',
 'Knowledge engineering',
 'Personal knowledge management',
 'Knowledge value chain',
 'Transparency (graphic)',
 'Usability',
 'Information retr

In [174]:
classes_with_names.loc[1]

['Graph',
 'Combinatorics',
 'Algebra',
 'Theoretical computer science',
 'Decidability',
 'Decision problem',
 'Undecidable problem',
 'Nondeterministic algorithm',
 'Bounded function',
 'Cardinality',
 'Integer',
 'Topology',
 'Algorithm',
 'Programming language',
 'Sorting',
 'Computation',
 'Vertex (geometry)',
 'Time complexity',
 'Completeness (statistics)',
 'Line segment',
 'Polyhedron',
 'Convex hull',
 'Mesh generation',
 'Graph theory',
 'Binary code',
 'Polynomial',
 'Upper and lower bounds',
 'Triangulation (social science)',
 'Implementation',
 'Operator (computer programming)',
 'Algorithm design',
 'Lattice (order)',
 'Generalization',
 'Meta-optimization',
 'Metaheuristic',
 'Multi-swarm optimization',
 'Genetic algorithm',
 'Particle swarm optimization',
 'Local search (optimization)',
 'Hybrid algorithm',
 'Spin-½',
 'Parameterized complexity',
 'Partition (number theory)',
 'Set theory',
 'Fifth-generation programming language',
 'High-level programming language',
 

In [175]:
classes_with_names.loc[2]

['Network architecture',
 'Computer network',
 'Keying',
 'Architecture',
 'Modulation',
 'Access network',
 'Wavelength-division multiplexing',
 'Multiplexing',
 'Broadcasting',
 'Ubiquitous computing',
 'Verifiable secret sharing',
 'Secret sharing',
 'Latency (engineering)',
 'Wireless sensor network',
 'Wi-Fi array',
 'Wireless network',
 'Power control',
 'Channel capacity',
 'Communication channel',
 'Mobile telephony',
 'Mobile computing',
 'Middleware',
 'Mobile phone',
 'Link-state routing protocol',
 'Static routing',
 'Wireless Routing Protocol',
 'Multipath routing',
 'Policy-based routing',
 'Dynamic Source Routing',
 'Equal-cost multi-path routing',
 'Hash function',
 'Computer security',
 'Internet privacy',
 'Grid',
 'Redundancy (engineering)',
 'Transmission (mechanics)',
 'Information theory',
 'Concatenated error correction code',
 'Decoding methods',
 'Mobile technology',
 'Provisioning',
 'The Internet',
 'Scheduling (computing)',
 'Scalability',
 'Detector',
 'Err

In [176]:
classes_with_names.loc[3]

['Matrix multiplication',
 'Electronic engineering',
 'Electronic circuit',
 'Low voltage',
 'Leakage (electronics)',
 'Transistor',
 'CMOS',
 'Message passing',
 'Composite material',
 'Parallel computing',
 'Computer architecture',
 'Analytical chemistry',
 'Dataflow',
 'Circuit design',
 'Integrated circuit',
 'Computational science',
 'Fortran',
 'Voltage',
 'Thread (computing)',
 'Multi-core processor',
 'Low-power electronics',
 'Sequential logic',
 'Logic synthesis',
 'Logic gate',
 'Integrated circuit design',
 'Concurrent computing',
 'Phase-locked loop',
 'Macro',
 'Chip',
 'Computer hardware',
 'Memory management',
 'Cache',
 'Microarchitecture',
 'Materials science',
 'Digital electronics',
 'Digital signal processing',
 'Nanotechnology',
 'System on a chip',
 'Code generation',
 'Multiprocessing',
 'Lookup table',
 'Microcontroller',
 'Threshold voltage',
 'Dielectric',
 'Application-specific integrated circuit',
 'Field-programmable gate array',
 'Very-large-scale integra

In [177]:
classes_with_names.loc[4]

['Quality assurance',
 'Application software',
 'Metamodeling',
 'Systems engineering',
 'Reuse',
 'Computer engineering',
 'Software design pattern',
 'Engineering design process',
 'Software development',
 'Design process',
 'Software maintenance',
 'Software metric',
 'Software system',
 'Software quality',
 'Object-oriented programming',
 'Modeling and simulation',
 'Software',
 'Source code',
 'Reliability engineering',
 'Java',
 'Software construction',
 'Software requirements specification',
 'Software development process',
 'Software engineering',
 'Social software engineering',
 'Software design',
 'Automation',
 'Agile software development',
 'Object-oriented design',
 'Software architecture',
 'Software peer review',
 'System of systems',
 'Component-based software engineering',
 'Software project management',
 'Dependability',
 'Complex system',
 'Systems design',
 'Reusability',
 'Documentation',
 'Test suite',
 'Reference architecture',
 'Resource-oriented architecture',


In [178]:
classes_with_names.loc[5]

['Defuzzification',
 'Fuzzy set operations',
 'Fuzzy logic',
 'Fuzzy number',
 'Type-2 fuzzy sets and systems',
 'Fuzzy mathematics',
 'Fuzzy classification',
 'Fuzzy control system',
 'Fuzzy set',
 'Soft computing',
 'Neuro-fuzzy',
 'Membership function',
 'Fuzzy associative matrix',
 'Fuzzy rule',
 'Adaptive neuro fuzzy inference system',
 'Fuzzy measure theory']

In [179]:
classes_with_names.loc[6]

['Image processing',
 'Hidden Markov model',
 'Inertial measurement unit',
 'Physical medicine and rehabilitation',
 'Rehabilitation',
 'Gait',
 'Matrix (mathematics)',
 'Speech recognition',
 'Magnetic resonance imaging',
 'Engineering',
 'Applied mathematics',
 'Simulation',
 'Robot',
 'Motion estimation',
 'Eye tracking',
 'GRASP',
 'Stereopsis',
 'Input/output',
 'Sampling (statistics)',
 'Control theory',
 'Control engineering',
 'Adaptive control',
 'Estimator',
 'Collision',
 'Linear subspace',
 'Humanoid robot',
 'Social robot',
 'Robot control',
 'Human–robot interaction',
 'Motion capture',
 'Ordinary differential equation',
 'Steady state',
 'Differential equation',
 'Deconvolution',
 'Optics',
 'Wavelength',
 'Artificial neural network',
 'Calculus',
 'Pi',
 'Computational physics',
 'Beam (structure)',
 'Experimental data',
 'Physics',
 'Remote sensing',
 'Radar imaging',
 'Iterative reconstruction',
 'Inverse synthetic aperture radar',
 'Synthetic aperture radar',
 'Image

In [180]:
classes_with_names.loc[7]

['Gene regulatory network',
 'Gene expression',
 'Chemistry',
 'Protein structure',
 'Bioinformatics',
 'Computational chemistry',
 'Molecule',
 'Gene expression profiling',
 'Gene',
 'Biology',
 'Computational biology',
 'Genetics',
 'Genome',
 'Molecular biology',
 'DNA microarray',
 'Microarray analysis techniques',
 'Phylogenetic tree',
 'DNA',
 'Genomics',
 'DNA sequencing',
 'Sequence alignment',
 'Ecology',
 'Cell biology',
 'Systems biology',
 'Molecular dynamics',
 'Cancer']

In [181]:
classes_with_names.loc[8]

['Scheduling (production processes)',
 'Flow shop scheduling',
 'Fair-share scheduling',
 'Round-robin scheduling',
 'Two-level scheduling',
 'Fixed-priority pre-emptive scheduling',
 'Rate-monotonic scheduling',
 'Dynamic priority scheduling',
 'Earliest deadline first scheduling']

In [186]:
classes.to_csv("classes_fos.csv")

In [187]:
classes_with_names.to_csv("classes_fos_names.csv")

In [195]:
scheduling_class = classes[classes["modularity_class"] == 8]["Id"].values

In [196]:
scheduling_class

array([2639, 1368,  193,  689,  191,  199,  196,  198, 3850])

Read in publications from scheduling class.

In [239]:
def common_member(list_a, list_b):
    a_set = set(list_a)
    b_set = set(list_b)
    return len(a_set & b_set) > 0

def reader_fos(fileName, foses_class):
    df = pandas.read_json(fileName, lines=True)
    fos_with_nan_and_wages = df[["fos", "authors", "year"]]
    fos_since_1990 = fos_with_nan_and_wages.loc[fos_with_nan_and_wages["year"] > 2010]
    fos_with_wages = fos_since_1990.loc[fos_since_1990["fos"].apply(lambda x: type(x) is not float)]
    fos = fos_with_wages
    fos["fos"] = fos_with_wages["fos"].apply(lambda row: [int(field["name"]) for field in row])
    fos_from_class = fos.loc[fos["fos"].apply(lambda foses: common_member(foses, foses_class))]
    return fos_from_class["authors"]

In [240]:
dblp_part_regex = re.compile("author_indexed_fos*")
part_files_filter_obj = filter(lambda file_name: dblp_part_regex.match(file_name), onlyfiles)

In [241]:
part_files = list(part_files_filter_obj)

In [242]:
list_of_dataframes = list(map(lambda fileName: reader_fos(fileName, scheduling_class), part_files))

In [243]:
scheduling_df = pandas.concat(list_of_dataframes, ignore_index=True)

In [244]:
scheduling_df

0                    [{'index': 517355}, {'index': 37954}]
1                   [{'index': 517365}, {'index': 517366}]
2        [{'index': 518019}, {'index': 35835}, {'index'...
3                   [{'index': 518099}, {'index': 518100}]
4        [{'index': 519021}, {'index': 519022}, {'index...
                               ...                        
10009    [{'index': 26213}, {'index': 364797}, {'index'...
10010    [{'index': 365116}, {'index': 252447}, {'index...
10011    [{'index': 238274}, {'index': 128167}, {'index...
10012                                  [{'index': 366577}]
10013               [{'index': 366621}, {'index': 366622}]
Name: authors, Length: 10014, dtype: object

In [245]:
withoutOneAuthors = scheduling_df.loc[scheduling_df.apply(len) > 1]

In [246]:
authors = withoutOneAuthors

Create graph from them.

In [247]:
authors_graph = {}
import itertools
def add_authors_to_graph(authors_list, graph):
    for x, y in itertools.product(authors_list, authors_list):
        x_index = x["index"]
        y_index = y["index"]
        if x_index != y_index:
            if x_index < y_index:
                edge_name = str(x_index) + "_" + str(y_index)
            else:
                edge_name = str(y_index) + "_" + str(x_index)
            weight = graph.get(edge_name, 0)
            graph[edge_name] = weight + 1
            
authors.apply(lambda authors_group: add_authors_to_graph(authors_group, authors_graph))

0        None
1        None
2        None
3        None
4        None
         ... 
10008    None
10009    None
10010    None
10011    None
10013    None
Name: authors, Length: 9464, dtype: object

In [248]:
len(authors_graph)

41262

In [249]:
authors_ngraph = networkx.Graph()
for edge, weight in authors_graph.items():
    nodes = edge.split("_")
    node_x = int(nodes[0])
    node_y = int(nodes[1])
    authors_ngraph.add_edge(node_x, node_y, weight = weight)

In [250]:
networkx.write_gexf(authors_ngraph, "schedulers_authors_graph.gexf")

Export to file. Analysis in Gephi.